In [26]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from sodapy import Socrata
import datetime

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

<IPython.core.display.Javascript object>

In [2]:
%load_ext nb_black
%matplotlib inline


<IPython.core.display.Javascript object>

In [3]:
CODE_INTEGRATED = "rpmr-utcd"
CODE_SECOPII = "p6dx-8zbt"


<IPython.core.display.Javascript object>

In [5]:
client = Socrata("www.datos.gov.co", None)
request = client.get(CODE_SECOPII, limit=1000, select="distinct nit_entidad")
nits_list = [x["nit_entidad"] for x in request]
{x: {"req": 0, "date": 0} for x in nits_list}

[07/13/22 16:25:13] WARNING  Requests made without an app_token will be subject to strict throttling  ]8;id=792682;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/sodapy/socrata.py\socrata.py]8;;\:]8;id=1158;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/sodapy/socrata.py#63\63]8;;\
                             limits.                                                                               

<IPython.core.display.Javascript object>

In [4]:
client = Socrata("www.datos.gov.co", None)

[07/13/22 17:03:33] WARNING  Requests made without an app_token will be subject to strict throttling  ]8;id=203417;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/sodapy/socrata.py\socrata.py]8;;\:]8;id=948425;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/sodapy/socrata.py#63\63]8;;\
                             limits.                                                                               

<IPython.core.display.Javascript object>

In [5]:
secop2_log = catalog.load('secop2_log_in')

[07/13/22 17:05:21] INFO     Loading data from 'secop2_log_in' (JSONDataSet)...                 ]8;id=508391;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=64419;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'000000001'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
nits_to_extract = [k for k in secop2_log.keys() if secop2_log[k]["req"] == 0]
if len(nits_to_extract) > 0:
    nit_to_extract = nits_to_extract[0]
else:
    list_dates = [(secop2_log[k]["date"], k) for k in secop2_log.keys()]
    list_dates.sort()
    nit_to_extract = list_dates[0][1]
client = Socrata("www.datos.gov.co", None)
lim = 2000
offset = lim
print(f"req - {offset-lim} - {datetime.datetime.now()}")
request = client.get(
    CODE_SECOPII, limit=lim, where='nit_entidad = "' + nit_to_extract + '"'
)
request_df = pd.DataFrame.from_records(request)
results_df = request_df.copy()
while len(request_df) > 0:
    print(f"req - {offset} - {datetime.datetime.now()}")
    request = client.get(
        CODE_SECOPII,
        limit=lim,
        offset=offset,
        where='nit_entidad = "' + nit_to_extract + '"',
    )
    request_df = pd.DataFrame.from_records(request)
    results_df = pd.concat([results_df, request_df], ignore_index=True)
    offset += lim
    
secop2_log[nit_to_extract]["req"] = 1
secop2_log[nit_to_extract]["date"] = str(datetime.datetime.now())
spark = SparkSession.builder.getOrCreate()
sql_ctx = SQLContext(spark.sparkContext)
#return secop2_log, sql_ctx.createDataFrame(results_df)

[07/13/22 17:14:12] WARNING  Requests made without an app_token will be subject to strict throttling  ]8;id=934206;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/sodapy/socrata.py\socrata.py]8;;\:]8;id=152846;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/sodapy/socrata.py#63\63]8;;\
                             limits.                                                                               

req - 0 - 2022-07-13 17:14:12.366154
req - 2000 - 2022-07-13 17:14:13.784813


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
results_df[[c for c in results_df.columns if "fecha" in c]] = results_df[
    [c for c in results_df.columns if "fecha" in c]
].fillna("")

<IPython.core.display.Javascript object>

In [29]:
results_df["fecha_de_publicacion_del"].fillna("")

0                           
1                           
2    2021-04-26T00:00:00.000
3    2021-04-27T00:00:00.000
Name: fecha_de_publicacion_del, dtype: object

<IPython.core.display.Javascript object>

In [41]:
results_df.columns

Index(['entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad',
       'ordenentidad', 'codigo_pci', 'id_del_proceso',
       'referencia_del_proceso', 'ppi', 'id_del_portafolio',
       'nombre_del_procedimiento', 'descripci_n_del_procedimiento', 'fase',
       'precio_base', 'modalidad_de_contratacion',
       'justificaci_n_modalidad_de', 'duracion', 'unidad_de_duracion',
       'ciudad_de_la_unidad_de', 'nombre_de_la_unidad_de',
       'proveedores_invitados', 'proveedores_con_invitacion',
       'visualizaciones_del', 'proveedores_que_manifestaron',
       'respuestas_al_procedimiento', 'respuestas_externas',
       'conteo_de_respuestas_a_ofertas', 'proveedores_unicos_con',
       'numero_de_lotes', 'estado_del_procedimiento',
       'id_estado_del_procedimiento', 'adjudicado', 'id_adjudicacion',
       'codigoproveedor', 'departamento_proveedor', 'ciudad_proveedor',
       'valor_total_adjudicacion', 'nombre_del_adjudicador',
       'nombre_del_proveedor', 'nit_del_proveedor_adjudicado',
       'codigo_principal_de_categoria', 'estado_de_apertura_del_proceso',
       'tipo_de_contrato', 'subtipo_de_contrato', 'categorias_adicionales',
       'codigo_entidad', 'estadoresumen', 'fecha_de_publicacion_del',
       'fecha_de_ultima_publicaci', 'fecha_de_publicacion_fase_3',
       'fecha_de_recepcion_de', 'fecha_de_apertura_efectiva', 'urlproceso'],
      dtype='object')

<IPython.core.display.Javascript object>

In [50]:
results_df.iloc[0]

entidad                                                 Vortal Operations
nit_entidad                                                     000000001
departamento_entidad                                              Quindío
ciudad_entidad                                                    Armenia
ordenentidad                                                  Territorial
codigo_pci                                                             Si
id_del_proceso                                             CO1.REQ.948804
referencia_del_proceso                                     CMA_MFR_2808_2
ppi                                                                    ND
id_del_portafolio                                         CO1.BDOS.915520
nombre_del_procedimiento                                       CMAMFR2808
descripci_n_del_procedimiento                                  CMAMFR2808
fase                                                          No Definido
precio_base                                                            -1
modalidad_de_contratacion         Concurso de méritos con precalificación
justificaci_n_modalidad_de                               Ley 1150 de 2007
duracion                                                                0
unidad_de_duracion                                                     ND
ciudad_de_la_unidad_de                                        No definida
nombre_de_la_unidad_de                                               CC01
proveedores_invitados                                                   0
proveedores_con_invitacion                                              0
visualizaciones_del                                                     0
proveedores_que_manifestaron                                            0
respuestas_al_procedimiento                                             0
respuestas_externas                                                     0
conteo_de_respuestas_a_ofertas                                          0
proveedores_unicos_con                                                  0
numero_de_lotes                                                         0
estado_del_procedimiento                                      No Definido
id_estado_del_procedimiento                                           110
adjudicado                                                             No
id_adjudicacion                                             No Adjudicado
codigoproveedor                                             No Adjudicado
departamento_proveedor                                          No aplica
ciudad_proveedor                                                No aplica
valor_total_adjudicacion                                                0
nombre_del_adjudicador                                      No Adjudicado
nombre_del_proveedor                                        No Adjudicado
nit_del_proveedor_adjudicado                                No Adjudicado
codigo_principal_de_categoria                                 V1.10101500
estado_de_apertura_del_proceso                                    Abierto
tipo_de_contrato                                              Consultoría
subtipo_de_contrato                                       No Especificado
categorias_adicionales                                                 ND
codigo_entidad                                                  700003015
estadoresumen                                                 No Definido
fecha_de_publicacion_del                                                 
fecha_de_ultima_publicaci                                                
fecha_de_publicacion_fase_3                                              
fecha_de_recepcion_de                                                    
fecha_de_apertura_efectiva                                               
urlproceso                                                            NaN
Name: 0, dtype: object

<IPython.core.display.Javascript object>

In [51]:
results_df["proveedores_con_invitacion"].dtype

dtype('O')

<IPython.core.display.Javascript object>

In [52]:
results_df.dtypes

entidad                           object
nit_entidad                       object
departamento_entidad              object
ciudad_entidad                    object
ordenentidad                      object
codigo_pci                        object
id_del_proceso                    object
referencia_del_proceso            object
ppi                               object
id_del_portafolio                 object
nombre_del_procedimiento          object
descripci_n_del_procedimiento     object
fase                              object
precio_base                       object
modalidad_de_contratacion         object
justificaci_n_modalidad_de        object
duracion                          object
unidad_de_duracion                object
ciudad_de_la_unidad_de            object
nombre_de_la_unidad_de            object
proveedores_invitados             object
proveedores_con_invitacion        object
visualizaciones_del               object
proveedores_que_manifestaron      object
respuestas_al_procedimiento       object
respuestas_externas               object
conteo_de_respuestas_a_ofertas    object
proveedores_unicos_con            object
numero_de_lotes                   object
estado_del_procedimiento          object
id_estado_del_procedimiento       object
adjudicado                        object
id_adjudicacion                   object
codigoproveedor                   object
departamento_proveedor            object
ciudad_proveedor                  object
valor_total_adjudicacion          object
nombre_del_adjudicador            object
nombre_del_proveedor              object
nit_del_proveedor_adjudicado      object
codigo_principal_de_categoria     object
estado_de_apertura_del_proceso    object
tipo_de_contrato                  object
subtipo_de_contrato               object
categorias_adicionales            object
codigo_entidad                    object
estadoresumen                     object
fecha_de_publicacion_del          object
fecha_de_ultima_publicaci         object
fecha_de_publicacion_fase_3       object
fecha_de_recepcion_de             object
fecha_de_apertura_efectiva        object
urlproceso                        object
dtype: object

<IPython.core.display.Javascript object>

In [ ]:
si
sisisuausdfasiudfhas